## Lets use the embeddings generated from [1] and prepare lots of data for analysis in [3]

In [1]:
# Standard library imports
import glob
import json
import os
import shutil

# Related third-party imports
import numpy as np
import pandas as pd
import glob
import pacmap
from matplotlib import pyplot as plt
import seaborn as sns
import torch
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor
import optuna

# Local application/library specific imports
from utils.standard_ds_loader import StandardDatasetLoader

/home/zbates/envs/hyena-dna/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Create and check directories. You will be able to make embeddings from a model with/without fine-tuning, and also be able to add additional features that resemble the presence of various important sequences in 3' end**

In [8]:
directories = [
    './hyena_embeds',
    './hyena_embeds_pe',
    './fine_tuned_hyena_embeds',
    './fine_tuned_hyena_embeds_pe'
]

subdirs = ['kfold_results', 'kfold_plots']

# Checks if the directories and subdirectories exist, and creates them if they don't
for dir in directories:
    if not os.path.exists(dir):
        os.makedirs(dir)
    for subdir in subdirs:
        full_path = os.path.join(dir, subdir)
        if not os.path.exists(full_path):
            os.makedirs(full_path)
        assert os.path.exists(full_path), f"Failed to create {full_path}"

smart_read_csv = StandardDatasetLoader().smart_read_csv

def smart_read_embeddings(filepath):
    _, ext = os.path.splitext(filepath)
    print(filepath)
    # Check the file extension and use the appropriate library to load the data
    if ext in ['.pt', '.pth']:
        # Load PyTorch tensor
        data = torch.load(filepath)
        if isinstance(data, torch.Tensor):
            return data.shape[0]
        else:
            raise ValueError(f"File {filepath} does not contain a PyTorch tensor.")
    elif ext == '.npy':
        # Load Numpy array
        data = np.load(filepath)
        return data.shape[0]
    else:
        raise ValueError(f"Unsupported file extension {ext}. Supported extensions are .pt, .pth, and .npy.")

def test_X_y_paths(non_tuned_embeddings_path, tuned_embeddings_path, label_data_path):
    non_tuned_X_path = non_tuned_embeddings_path
    tuned_X_path = tuned_embeddings_path
    target_values_path = label_data_path
    
    target_df = smart_read_csv(target_values_path)
    assert target_df.shape[1] == 2, f"Wrong number of columns in target_values_path: {target_df.shape[1]}, should be 2, with column one containing the sequences, and column two containing the labels."
    df_length = len(target_df)
    
    # Getting the shape of the first dimension for embeddings
    non_tuned_length = smart_read_embeddings(non_tuned_X_path)
    tuned_length = smart_read_embeddings(tuned_X_path)
    
    # Now you can compare or do any other operations with df_length, non_tuned_length, and tuned_length
    # For instance:
    assert df_length == non_tuned_length == tuned_length, f"Mismatching lengths between dataframes: {df_length}, {non_tuned_length}, {tuned_length}, this means you didn't load the correct embeddings and their corresponding labels. You probably didn't load the updated datasets."


# Load your embeddings (X) paths, label (y) paths, and Scikit-Learn settings

**The embeddings were generated using a genomics foundational model. However, we did not use the neural net to infer on the sequences. This is simply a information-rich way of representing gene sequences, and a more accurate one than traditional one-hot-encoding.**

In [3]:
#=================================Load a couple parameters==============================

# Start off by instantiating the StandardDatasetLoader
# Select your dataset here. You can choose between 'standard' or path a datapath to your own custom dataset. The 'standard' option will load the dataset selected in [1].
dataset = 'standard' # or pass a path to your custom dataset here
standard_ds_loader = StandardDatasetLoader()
PATH_TO_GENERATED_EMBEDDINGS = os.path.expanduser(standard_ds_loader.path_to_generated_embeddings)

# Creating a function to load the hyena data process settings simply
def dataset_creator(dataset):
    
    if dataset == 'standard':
        n_folds, model_dict, model_list, SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING, FINE_TUNED_SEQS_EMBEDDINGS_PATH, OPTIMAL_SEQS_DATA_PATH, LABEL_DATA_PATH = standard_ds_loader.load_hyena_data_process_settings()

    else:
        print('Detected a custom datapath entry, will try loading using smart_read_csv')
        LABEL_DATA_PATH = dataset
        n_folds, model_dict, model_list, SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING, FINE_TUNED_SEQS_EMBEDDINGS_PATH, OPTIMAL_SEQS_DATA_PATH, _ = standard_ds_loader.load_hyena_data_process_settings()        

    return n_folds, model_dict, model_list, SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING, FINE_TUNED_SEQS_EMBEDDINGS_PATH, OPTIMAL_SEQS_DATA_PATH, LABEL_DATA_PATH

# Call the function
n_folds, model_dict, model_list, SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING, FINE_TUNED_SEQS_EMBEDDINGS_PATH, OPTIMAL_SEQS_DATA_PATH, LABEL_DATA_PATH = dataset_creator(dataset)

assert os.path.exists(SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING) and os.path.exists(FINE_TUNED_SEQS_EMBEDDINGS_PATH), f'Embedding paths {SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING} and {FINE_TUNED_SEQS_EMBEDDINGS_PATH} do not exist'
assert os.path.exists(LABEL_DATA_PATH), f'Label data path {LABEL_DATA_PATH} does not exist'

**Lets test that we loaded the correct X embeddings (fine-tuned and not fine-tuned) and target values**

**It will test the length of these to make sure you have the correct ds loaded**

In [5]:
test_X_y_paths(SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING, FINE_TUNED_SEQS_EMBEDDINGS_PATH, LABEL_DATA_PATH)

./data/ds_fine_tuned_embeddings/mmseqs_train_1.pth
./data/ds_fine_tuned_embeddings/mmseqs_train_6.pth


In [6]:
def process_and_save_embeddings(dataset):

    if dataset == 'UTR_regions':
        file_path = SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING
        directory_name = './hyena_embeds_pe'
    elif dataset == 'no_augment':
        file_path = SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING
        directory_name = './hyena_embeds'
    elif dataset == 'fine_tuned':
        file_path = FINE_TUNED_SEQS_EMBEDDINGS_PATH
        directory_name = './fine_tuned_hyena_embeds'
    elif dataset == 'fine_tuned_pe':
        file_path = FINE_TUNED_SEQS_EMBEDDINGS_PATH
        directory_name = './fine_tuned_hyena_embeds_pe'
    else:
        raise ValueError(f'Invalid dataset: {dataset}: must be one of [UTR_regions, no_augment, fine_tuned, fine_tuned_pe]')
    
    embeddings = torch.load(file_path)
    # Push to CPU
    embeddings = embeddings.cpu()
    
    # # Convert to numpy
    # embeddings = embeddings.numpy()
    
    #     # If it's already a numpy array, proceed directly to reshaping
    # if isinstance(embeddings, np.ndarray):
    #     X_reshaped = embeddings.reshape(len(embeddings), -1)
    #     print('X shape:', X_reshaped.shape)


    # Calculate the average along the last dimension (128)
    average_array = torch.mean(embeddings, dim=2).numpy()

    # Calculate the max and min along the last dimension (128)
    max_array = torch.max(embeddings, dim=2)[0].numpy()
    min_array = torch.min(embeddings, dim=2)[0].numpy()

    # Stack max and min arrays along the last dimension (2)
    max_min_array = np.stack([max_array, min_array], axis=2)

    # Reshape the max and min arrays to be 2D arrays
    min_max_array_reshaped = max_min_array.reshape(len(max_min_array), -1)

    # Print the shapes of the resulting NumPy arrays
    print("Average Array Shape:", average_array.shape)
    print("Max-Min Array Shape:", max_min_array.shape)
    print('min_max_array_reshaped shape:', min_max_array_reshaped.shape)

    # Create the directory if it doesn't already exist
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)

    # Save the reshaped arrays to .npy files
    np.save(f'{directory_name}/reshaped_min_max_hyena.npy', min_max_array_reshaped)
    np.save(f'{directory_name}/reshaped_average_hyena.npy', average_array)
    print(f"Embeddings saved to {directory_name}/")

# Example usage with shalem_embeddings and shalem_embeds as the directory name
# process_and_save_embeddings(dataset='UTR_regions')


***Let's do Dimensionality Reduction as well***

In [7]:
def load_and_scale_data(data):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    return scaled_data

def process_embeddings_if_needed(embeddings):
    # Check if embeddings is a PyTorch tensor, and if so, convert to NumPy array
    if isinstance(embeddings, torch.Tensor):
        # Push to the CPU if it's on a GPU
        if embeddings.is_cuda:
            embeddings = embeddings.cpu()
        
        # Convert to numpy array
        embeddings = embeddings.numpy()

    # If it's already a numpy array, proceed directly to reshaping
    if isinstance(embeddings, np.ndarray):
        X_reshaped = embeddings.reshape(len(embeddings), -1)
        print('X shape:', X_reshaped.shape)
        
    else:
        print("The provided embeddings are neither a PyTorch tensor nor a NumPy array.")

    return X_reshaped

METHODS = ['pca', 'pacmap']
COMPONENTS = [2, 5, 10, 150]

def perform_dimensionality_reduction(X_reshaped, base_dir, methods, components):
    for method in methods:
        for num_components in components:
            print('Method and number of components:', method, num_components)

            if method == 'pca':
                file_name = os.path.join(base_dir, f'pca_components_{num_components}.npy')
            elif method == 'pacmap':
                if num_components == 150:
                    num_components = 100
                file_name = os.path.join(base_dir, f'pacmap_components_{num_components}.npy')
            else:
                print(f"Unsupported method: {method}")
                continue

            if os.path.exists(file_name):
                print(f"File {file_name} already exists. Skipping.")
                continue

            X_reshaped_scaled = StandardScaler().fit_transform(X_reshaped)

            if method == 'pca':
                dim_reduct = PCA(n_components=num_components)
            elif method == 'pacmap':
                # Make sure to import pacmap before using
                dim_reduct = pacmap.PaCMAP(n_components=num_components, n_neighbors=7)
            
            X_reduced = dim_reduct.fit_transform(X_reshaped_scaled)
            assert X_reduced.shape[1] == num_components, f'Expected {num_components} components, but got {X_reduced.shape[1]}'

            print('Shape of X array after reduction:', X_reduced.shape)
            assert not np.isnan(X_reduced).any(), f'The numpy array contains NaNs'
            
            np.save(file_name, X_reduced)
            print(f"Reduced data saved to {file_name}")

# Example usage, replace 'your_X_reshaped' with your actual reshaped data
# perform_dimensionality_reduction(your_X_reshaped)

In [9]:
# Call the function on shalem_embeds to perform dimensionality reduction
non_fine_tuned_embeddings = torch.load(SEQS_EMBEDDINGS_PATH_NO_FINE_TUNING)
process_and_save_embeddings(dataset='no_augment')
X_reshaped = process_embeddings_if_needed(non_fine_tuned_embeddings)
perform_dimensionality_reduction(X_reshaped, base_dir='./hyena_embeds', methods=METHODS, components=COMPONENTS)

Average Array Shape: (10447, 152)
Max-Min Array Shape: (10447, 152, 2)
min_max_array_reshaped shape: (10447, 304)
Embeddings saved to ./hyena_embeds/
X shape: (10447, 19456)
Method and number of components: pca 2
Shape of X array after reduction: (10447, 2)
Reduced data saved to ./hyena_embeds/pca_components_2.npy
Method and number of components: pca 5
Shape of X array after reduction: (10447, 5)
Reduced data saved to ./hyena_embeds/pca_components_5.npy
Method and number of components: pca 10
Shape of X array after reduction: (10447, 10)
Reduced data saved to ./hyena_embeds/pca_components_10.npy
Method and number of components: pca 150
Shape of X array after reduction: (10447, 150)
Reduced data saved to ./hyena_embeds/pca_components_150.npy
Method and number of components: pacmap 2
Shape of X array after reduction: (10447, 2)
Reduced data saved to ./hyena_embeds/pacmap_components_2.npy
Method and number of components: pacmap 5
Shape of X array after reduction: (10447, 5)
Reduced data s

***It's likely that the strong terminators have a Positioning Element (pe), Efficiency Element (EE), and PolyA Site (pas) within them. In the sequences, lets check this by creating more features, denoting the presence of these unique sequences***

***You likely don't need this, as I don't see much of a performance boost, but this pipeline can be edited easily if you want to use a more sophisticated scoring technique***

In [10]:
def produce_optimal_seq_dataset(target_df_path, path_save_opt_seqs):
    # Define the optimal sequences found in the 3' region of yeast genes, that are associated with highly expressed transcripts
    optimal_sequences = ['TATATA', 'AATAAA', 'CAAA']
    
    
    df_opt_seqs = smart_read_csv(target_df_path)
    # Add column names for shalem_df
    df_opt_seqs.columns = ['sequence', 'tpm']
    print('Head of ds: ', df_opt_seqs.head())


    df_opt_seqs['ee'] = 0
    df_opt_seqs['pe'] = 0
    df_opt_seqs['pas'] = 0

    for index, row in df_opt_seqs.iterrows():
        sequence = row['sequence']
        for i, optimal in enumerate(optimal_sequences):
            if optimal in sequence:
                if i == 0:
                    df_opt_seqs.at[index, 'ee'] = 1
                elif i == 1:
                    df_opt_seqs.at[index, 'pe'] = 1
                elif i == 2:
                    df_opt_seqs.at[index, 'pas'] = 1
    print('Head of df_opt_seqs: ', df_opt_seqs.head())

    # Save the updated dataframe to the file
    df_opt_seqs.to_csv(path_save_opt_seqs, sep="\t")
                        
#============Calling the function==================
produce_optimal_seq_dataset(target_df_path=LABEL_DATA_PATH, path_save_opt_seqs=OPTIMAL_SEQS_DATA_PATH)
#==================================================

# Create a function that will add the EE, PE and PAS columns to any loaded embedding
def stack_element_features(embedding, optimal_seqs_filepath):
    # Run the produce_optimal_seq_dataset function
    produce_optimal_seq_dataset(target_df_path=LABEL_DATA_PATH, path_save_opt_seqs=optimal_seqs_filepath)
    
    df = smart_read_csv(optimal_seqs_filepath)
    df_opt_seqs = df.iloc[:, 3:]
    print('Head of df_opt_seqs: ', df_opt_seqs.head())
    
    # Assert there there is only three columns, ee, pe and pas
    assert df_opt_seqs.shape[1] == 3, f'The df_opt_seqs is not comprised of three columns. It is supposed to be ee, pe and pas (3). df_opt_seqs: {df_opt_seqs.shape[1]}'
    assert (df_opt_seqs.dtypes == 'int64').all(), f'The df_opt_seqs is not comprised of integer values. It is supposed to be 1 or 0. df_opt_seqs: {df_opt_seqs.dtypes}'
    
    # Convert to numpy array
    opt_seqs_numpy = df_opt_seqs.to_numpy()
    
    additional_features_embeddings = np.hstack((embedding, opt_seqs_numpy))
    
    print('Shape of additional_features_embeddings: ', additional_features_embeddings.shape)
    print('Shape of embedding: ', embedding.shape)
    print('Shape of opt_seqs_numpy: ', opt_seqs_numpy.shape)
    
    return additional_features_embeddings

Head of ds:                                              sequence       tpm
0  GGGGACCAGGTGCCGTAAGGTGCGGCTGGCCCAATGTGCGCCTATG... -1.226293
1  GGGGACCAGGTGCCGTAAGCCATGAATTAATGAATATCTTTACTTA...  0.086170
2  GGGGACCAGGTGCCGTAAGGTCACGCTTACATTCACGCCCTCCTCC... -1.842423
3  GGGGACCAGGTGCCGTAAGTTGCTATGCAGATGCTTTATACTTCTT...  0.371952
4  GGGGACCAGGTGCCGTAAGCATTGTTAGCTTTTTATGCATTATAAT... -1.729185
Head of df_opt_seqs:                                              sequence       tpm  ee  pe  pas
0  GGGGACCAGGTGCCGTAAGGTGCGGCTGGCCCAATGTGCGCCTATG... -1.226293   0   1    1
1  GGGGACCAGGTGCCGTAAGCCATGAATTAATGAATATCTTTACTTA...  0.086170   1   1    1
2  GGGGACCAGGTGCCGTAAGGTCACGCTTACATTCACGCCCTCCTCC... -1.842423   1   1    1
3  GGGGACCAGGTGCCGTAAGTTGCTATGCAGATGCTTTATACTTCTT...  0.371952   0   0    1
4  GGGGACCAGGTGCCGTAAGCATTGTTAGCTTTTTATGCATTATAAT... -1.729185   0   0    0


***Lets start defining functions for our 10-fold Cross Validation and Swarm/Scatter-plot generation***

In [12]:
def save_scatterplot(y_actual, y_predicted, model_name, file_name, file_basename, dataset):
    from sklearn.metrics import r2_score
    import seaborn as sns
    from sklearn.metrics import mean_squared_error
    

    mse_test = mean_squared_error(y_actual, y_predicted)
    pearson_r_test = np.corrcoef(np.array(y_actual).ravel(), np.array(y_predicted).ravel())[0, 1]
    r2 = 1 - (mse_test / np.var(np.array(y_actual)))
        
    # plot_title = f'Scatterplot for {model_name}/{num}-components - {method} Measured vs Predicted Values)\nR-squared: {r2:.2f}'
    plot_title = f'Scatterplot for {file_name}_{model_name}_{file_basename}: Measured vs Predicted Values)\nR-squared: {r2:.2f}\nPearson Correlation: {pearson_r_test:.2f}'
    
    # Create a DataFrame for plotting
    df = pd.DataFrame({'Measured Values': y_actual.ravel(), 'Predicted Values': y_predicted.ravel()})
    
    if file_basename == 'train':
        color = 'tab:orange'
    elif file_basename == 'test':
        color = 'tab:blue'

    # Create the scatterplot using Seaborn
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=df, x='Measured Values', y='Predicted Values', color=color, alpha=0.5)
    plt.title(plot_title)
    plt.grid(True)
    
    if dataset == 'UTR_regions':
        plot_filename = f'./hyena_embeds_pe/kfold_plots/{file_name}_{model_name}_{file_basename}.png'
    elif dataset == 'no_augment':
        # Define the file path and name where you want to save the plot
        plot_filename = f'./hyena_embeds/kfold_plots/{file_name}_{model_name}_{file_basename}.png'
    elif dataset == 'fine_tuned':
        plot_filename = f'./fine_tuned_hyena_embeds/kfold_plots/{file_name}_{model_name}_{file_basename}.png'
    elif dataset == 'fine_tuned_pe':
        plot_filename = f'./fine_tuned_hyena_embeds_pe/kfold_plots/{file_name}_{model_name}_{file_basename}.png'

    # Save the plot as an image file (you can specify the file format, e.g., PNG, JPEG)
    plt.savefig(plot_filename)

    # Close the plot to release resources (optional)
    plt.close()
    
# Use case example
# run_cross_validation(model_name_function, hyperparams, X, y, n_folds, file_name=file_name_no_extension, feature_engineering=False)
# save_scatterplot(y_train_folds_concat, y_pred_train_folds_concat, feature_engineering=feature_engineering, model_name=model_name_function, file_name =file_name, file_basename='train')

def run_cross_validation(model_name_function, hyperparams, X, y, n_folds, file_name, dataset):
    
    # Make sure to reshape the target, if not already in (1, n_samples) format
    if len(y.shape) == 1:
        y = y.reshape(-1, 1)
            
    if model_name_function == 'xgb':
        model = XGBRegressor(**hyperparams)
    elif model_name_function == 'lr':
        model = LinearRegression(**hyperparams)
    else:
        raise ValueError("Invalid model name, choose from xgb, lr")
    
    # ========================MAKING ADJUSTMENTS TO SCALING PROCESS TO ENSURE NO DATA LEAKAGE!!!!==================
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
   
    y = scaler_y.fit_transform(y)


    # Create lists to store r2_train, r2_test, y_pred_train, and y_pred_test values for each fold
    r2_train_list = []
    r2_test_list = []

    # X_train_fold_list = []
    y_pred_train_fold_list = []
    y_train_fold_list = []
    # X_test_fold_list = []
    y_pred_test_fold_list = []
    y_test_fold_list = []

    RANDOM_STATE = 21
    # Perform K-fold cross-validation
    kf = KFold(n_splits=n_folds, random_state=RANDOM_STATE, shuffle=True)
    for train_index, test_index in kf.split(X):
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        
        print('Length of X-test fold: ', len(X_test_fold))
        print('Length of X-train fold: ', len(X_train_fold))
        
        # ==============================================USING NEW SCALING PROCESS=======================================
        X_train_fold = scaler_x.fit_transform(X_train_fold)
        X_test_fold = scaler_x.transform(X_test_fold)
        # =========================================================================================================
        
        model.fit(X_train_fold, y_train_fold)
        
        # Calculate R-squared on test set for this fold
        r2_test_fold = model.score(X_test_fold, y_test_fold)
        r2_test_list.append(r2_test_fold)
        
        # Calculate R-squared on train set for this fold
        r2_train_fold = model.score(X_train_fold, y_train_fold)
        r2_train_list.append(r2_train_fold)
        
        # Predict on the training set for this fold
        y_pred_train_fold = model.predict(X_train_fold)
        y_train_fold_list.append(y_train_fold)
        y_pred_train_fold_list.append(y_pred_train_fold)
        
        # Predict on the test set for this fold
        y_test_fold_list.append(y_test_fold)
        y_pred_test_fold = model.predict(X_test_fold)
        y_pred_test_fold_list.append(y_pred_test_fold)

    # Calculate the mean R-squared values for train and test sets
    mean_r2_train = np.mean(r2_train_list)
    mean_r2_test = np.mean(r2_test_list)

    # Concatenate the predicted values for training and testing sets
    y_pred_train_folds_concat = np.concatenate(y_pred_train_fold_list)
    y_train_folds_concat = np.concatenate(y_train_fold_list)
    y_pred_test_folds_concat = np.concatenate(y_pred_test_fold_list)
    y_test_folds_concat = np.concatenate(y_test_fold_list)
    print('Length of y_pred_train: ', len(y_pred_train_folds_concat))
    print('Length of y_pred_test: ', len(y_pred_test_folds_concat))

    # Create and save scatterplots for training and testing sets
    save_scatterplot(y_train_folds_concat, y_pred_train_folds_concat, model_name=model_name_function, file_name =file_name, file_basename='train', dataset=dataset)
    save_scatterplot(y_test_folds_concat, y_pred_test_folds_concat, model_name=model_name_function, file_name =file_name, file_basename='test', dataset=dataset)

    # Add the R-squared train/test list to df
    print('r2_train_list len:', len(r2_train_list))
    print('r2_test_list len:', len(r2_test_list))

    print('R2 Train List: ', r2_train_list)
    print('R2 Test List: ', r2_test_list)
    
    # Add the R-squared train/test list to df
    print('r2_train_list len:', len(r2_train_list))
    print('r2_test_list len:', len(r2_test_list))
    df_test = pd.DataFrame()
    df_train = pd.DataFrame()
    
    file_name = f'{file_name}_{model_name_function}'
    
    # Create the train R-squared column, and concatenate the test on axis 0 (vertical)
    df_test['r2'] = r2_train_list
    df_test['type'] = 'train'
    df_test['file_name'] = file_name

    df_train['r2'] = r2_test_list
    df_train['type'] = 'test'
    df_train['file_name'] = file_name
    
    # Concatenate the train and test sets (vertical)
    df = pd.concat([df_train, df_test], axis=0)
    assert len(df) == len(r2_test_list) + len(r2_train_list), f'The length of R2 df does not match the sum of r2_test_list ({len(r2_test_list)}) and r2_train_list ({len(r2_train_list)})'
        
    if dataset == 'UTR_regions':
        df.to_csv(f'./hyena_embeds_pe/kfold_results/R2_Values_{file_name}.csv', index=False)
    elif dataset == 'no_augment':
        df.to_csv(f'./hyena_embeds/kfold_results/R2_Values_{file_name}.csv', index=False)
    elif dataset == 'fine_tuned':
        df.to_csv(f'./fine_tuned_hyena_embeds/kfold_results/R2_Values_{file_name}.csv', index=False)
    elif dataset == 'fine_tuned_pe':
        df.to_csv(f'./fine_tuned_hyena_embeds_pe/kfold_results/R2_Values_{file_name}.csv', index=False)
    else:
        raise ValueError(f'Invalid dataset: {dataset}, choose from UTR_regions, no_augment, fine_tuned, fine_tuned_pe')


In [13]:
def glob_files(dataset):
    # Based on the dataset, get the list of reduced embeddings, for the 'pe' datasets, they will use their corresponding embeddings without the augmentation.
    # For example, the 'UTR_regions' dataset will use the non-fine-tuned embeddings, ./hyena_embeds/*.npy
    if dataset == 'UTR_regions':
        hyena_embeds_list = glob.glob(f'./hyena_embeds/*.npy')
    elif dataset == 'no_augment':
        hyena_embeds_list = glob.glob(f'./hyena_embeds/*.npy')
    elif dataset == 'fine_tuned':
        hyena_embeds_list = glob.glob(f'./fine_tuned_hyena_embeds/*.npy')
    elif dataset == 'fine_tuned_pe':
        hyena_embeds_list = glob.glob(f'./fine_tuned_hyena_embeds/*.npy')
    else:
        raise ValueError(f'Invalid dataset: {dataset}')
    
    hyena_embeds_list = [file for file in hyena_embeds_list if not os.path.isdir(file)]
    print('Hyena embeds list: ', hyena_embeds_list)
    
    return hyena_embeds_list


In [14]:
def run_cross_validation_for_files(hyena_embeds_list, model_list, model_dict, y, n_folds, dataset):
    for file_name in hyena_embeds_list:
        print('File Name: ', file_name)
        
        if not file_name.endswith(('.csv', '.npy')):
            print(f"Skipping invalid file type: {file_name}")
            continue

        file_name_no_extension = os.path.basename(file_name)[:-4]


        # Load X from the file
        if file_name.endswith('.csv'):
            X = pd.read_csv(file_name, header=None)
            assert not X.isnull().values.any(), f'NaN values in file: {file_name}'
        elif file_name.endswith('.npy'):
            X = np.load(file_name)
            assert not np.isnan(X).any(), f'NaN values in file: {file_name}'
        elif file_name.endswith('.pt') or file_name.endswith('.pth'):
            print(f"This is a torch file, and most likely a raw unprocessed embedding, skipping {file_name}.")
            continue
        
        # Lets concat the extra features is dataset 'UTR_regions' or 'fine_tuned_pe' is chosen
        if dataset == 'UTR_regions' or dataset == 'fine_tuned_pe':
            
            X = stack_element_features(embedding=X, optimal_seqs_filepath=OPTIMAL_SEQS_DATA_PATH)        

        print('File Name for validation: ', file_name_no_extension)
        assert os.stat(file_name).st_size > 0, f'Empty file: {file_name}'
        
        for model_name in model_list:
            print('Running model: ', model_name)
            hyperparams = model_dict[model_name]
            print('Model name and hyperparams: ', model_name, hyperparams)
            
            # Replace with actual function to run cross-validation
            run_cross_validation(model_name, hyperparams, X, y, n_folds, file_name=file_name_no_extension, dataset=dataset)

# Example usage (replace with actual values)
# run_cross_validation_for_files(hyena_embeds_list, model_list, model_dict, y, n_folds)

***Lets run the cross-validation for the hyena_embeds***


In [15]:
# Load target values
target_df = smart_read_csv(LABEL_DATA_PATH)
y = target_df.iloc[:, 1]
y_np = y.to_numpy()
print('y shape: ', y.shape)

y shape:  (10447,)


In [16]:
# Running the regular hyena-dna embeddings (no-fine-tuning but with no extra features)
hyena_embeds_list = glob_files('no_augment')
print('Hyena embeds list: ', hyena_embeds_list)

run_cross_validation_for_files(hyena_embeds_list, model_list, model_dict, y_np, n_folds, dataset='no_augment')

Hyena embeds list:  ['./hyena_embeds/reshaped_min_max_hyena.npy', './hyena_embeds/reshaped_average_hyena.npy', './hyena_embeds/pca_components_2.npy', './hyena_embeds/pca_components_5.npy', './hyena_embeds/pca_components_10.npy', './hyena_embeds/pca_components_150.npy', './hyena_embeds/pacmap_components_2.npy', './hyena_embeds/pacmap_components_5.npy', './hyena_embeds/pacmap_components_10.npy', './hyena_embeds/pacmap_components_100.npy']
Hyena embeds list:  ['./hyena_embeds/reshaped_min_max_hyena.npy', './hyena_embeds/reshaped_average_hyena.npy', './hyena_embeds/pca_components_2.npy', './hyena_embeds/pca_components_5.npy', './hyena_embeds/pca_components_10.npy', './hyena_embeds/pca_components_150.npy', './hyena_embeds/pacmap_components_2.npy', './hyena_embeds/pacmap_components_5.npy', './hyena_embeds/pacmap_components_10.npy', './hyena_embeds/pacmap_components_100.npy']
File Name:  ./hyena_embeds/reshaped_min_max_hyena.npy
File Name for validation:  reshaped_min_max_hyena
Running model:

Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1044
Length of X-train fold:  9403
Length of X-test fold:  1044
Length of X-train fold:  9403
Length of X-test fold:  1044
Length of X-train fold:  9403
Length of y_pred_train:  94023
Length of y_pred_test:  10447


/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.28536136026002623, 0.28565862202217396, 0.28321684578063233, 0.21206457766038522, 0.268459094907883, 0.27951154419559865, 0.2848752478782898, 0.281704316211405, 0.22407514383331506, 0.2844883259314097]
R2 Test List:  [0.19898566809211238, 0.20888976759032496, 0.24039538490967338, 0.23180757388027584, 0.18471298767236177, 0.22788692303874425, 0.22630766868699392, 0.2574252006957486, 0.171756592002979, 0.2279299033455986]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
L

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

...

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.8667593037187542, 0.8679912440171129, 0.86717460696773, 0.8684971405409477, 0.8749508151868692, 0.8762352384178796, 0.8717351759982044, 0.8702420845860759, 0.8700902156491591, 0.8746403840283306]
R2 Test List:  [0.30199760376611373, 0.3459316281115107, 0.31378989457321393, 0.3081077059385158, 0.33761153604577, 0.3020516929676864, 0.2744507947437108, 0.3329704649436571, 0.2743023438782093, 0.33288416402297993]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./hyena_embeds/pacmap_components_100.npy
File Name for validation:  pacmap_components_100
Running model:  lr
Model name and hyperparams:  lr {'fit_intercept': True}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of 

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.2611099582392029, 0.2600564354431987, 0.2588000416370101, 0.25069667049102173, 0.25626673961076996, 0.2529499228127283, 0.2601163789429346, 0.2597116293730408, 0.25851986071923505, 0.2554724623441109]
R2 Test List:  [0.2128855660396729, 0.21764975121509889, 0.23220705689751553, 0.3063216008931384, 0.25599957143224383, 0.28682432109095746, 0.22335800883213974, 0.22819998634326433, 0.23611592494935263, 0.2633926670128329]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
L

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9342053948859831, 0.9395959253064302, 0.9406731413580321, 0.9385329216938976, 0.9429823501485871, 0.9372834833249971, 0.9396859399505315, 0.9396253015122182, 0.9366460138596651, 0.9392114626932954]
R2 Test List:  [0.3219454718538898, 0.3598411767573525, 0.31264002187451834, 0.3539604176994193, 0.31157315558852794, 0.2826760512430838, 0.2460594977471754, 0.3367030947776394, 0.3146247913647676, 0.3385194995958315]
r2_train_list len: 10
r2_test_list len: 10


**Now lets reproduce this for the fine tuned embeddings**   

In [17]:
# Call the function on shalem_embeds to perform dimensionality reduction
fine_tuned_embeddings = torch.load(FINE_TUNED_SEQS_EMBEDDINGS_PATH)
X_reshaped = process_embeddings_if_needed(fine_tuned_embeddings)
process_and_save_embeddings(dataset='fine_tuned')
perform_dimensionality_reduction(X_reshaped, base_dir='./fine_tuned_hyena_embeds', methods=METHODS, components=COMPONENTS)

hyena_embeds_list = glob_files('fine_tuned')
print('Hyena embeds list: ', hyena_embeds_list)

# Running the fine-tuned hyena-dna embeddings with no extra features
run_cross_validation_for_files(hyena_embeds_list, model_list, model_dict, y_np, n_folds, dataset='fine_tuned')

X shape: (10447, 19456)
Average Array Shape: (10447, 152)
Max-Min Array Shape: (10447, 152, 2)
min_max_array_reshaped shape: (10447, 304)
Embeddings saved to ./fine_tuned_hyena_embeds/
Method and number of components: pca 2
Shape of X array after reduction: (10447, 2)
Reduced data saved to ./fine_tuned_hyena_embeds/pca_components_2.npy
Method and number of components: pca 5
Shape of X array after reduction: (10447, 5)
Reduced data saved to ./fine_tuned_hyena_embeds/pca_components_5.npy
Method and number of components: pca 10
Shape of X array after reduction: (10447, 10)
Reduced data saved to ./fine_tuned_hyena_embeds/pca_components_10.npy
Method and number of components: pca 150
Shape of X array after reduction: (10447, 150)
Reduced data saved to ./fine_tuned_hyena_embeds/pca_components_150.npy
Method and number of components: pacmap 2
Shape of X array after reduction: (10447, 2)
Reduced data saved to ./fine_tuned_hyena_embeds/pacmap_components_2.npy
Method and number of components: pa

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.32405546092863213, 0.3185367011256127, 0.3207301645262203, 0.3198242526307049, 0.32148004533986285, 0.32229214947264717, 0.32106761733777456, 0.32257108431917847, 0.3176317413189744, 0.31953261524887877]
R2 Test List:  [0.2416856246544261, 0.28504404551836793, 0.2607944078925586, 0.2802935926263863, 0.24471120280978598, 0.2586954504451454, 0.2679304381240565, 0.2648628096800155, 0.2919116772124076, 0.2861319888160796]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Len

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9991306502128569, 0.9990416634813737, 0.9990961333242288, 0.9991589570058809, 0.9990752429364089, 0.9990845316652985, 0.9990969255984614, 0.9990872962359518, 0.9990897313621461, 0.9991910631645029]
R2 Test List:  [0.43491435096381714, 0.48030871990692126, 0.4505528642552469, 0.448995795331101, 0.4478605019796035, 0.4510102474208558, 0.4199971487768426, 0.45057072618461913, 0.4301018155973365, 0.4814293426103864]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./fine_tuned_hyena_embeds/reshaped_average_hyena.npy
File Name for validation:  reshaped_average_hyena
Running model:  lr
Model name and hyperparams:  lr {'fit_intercept': True}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold: 

...

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.900877919171434, 0.89979020690936, 0.9058767890658267, 0.9057330827026329, 0.9000669197302358, 0.9060707687121675, 0.8998639679050457, 0.9034253978993855, 0.9038530604776271, 0.9016366397473244]
R2 Test List:  [0.42487134314713837, 0.4988115979093133, 0.4363311875218857, 0.4720564938634062, 0.4152451975110072, 0.44588945809205793, 0.47498214074045975, 0.481829214319471, 0.44451863529257296, 0.48642276177575694]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./fine_tuned_hyena_embeds/pacmap_components_100.npy
File Name for validation:  pacmap_components_100
Running model:  lr
Model name and hyperparams:  lr {'fit_intercept': True}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  94

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.41734629360971953, 0.41219827439359946, 0.4165670012264723, 0.4101874281123211, 0.41853003151204693, 0.4110843841330609, 0.41421141231220604, 0.41297822944456186, 0.41672624376809053, 0.40887314698759003]
R2 Test List:  [0.37333730066338344, 0.41789305014577227, 0.37818596960188067, 0.4380247178568458, 0.35775612065404916, 0.4295974859157774, 0.40293732385239633, 0.41369963676045085, 0.378762307362735, 0.44893342356909605]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  104

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9574998861144428, 0.9570122869988829, 0.9573990715583519, 0.955827498137855, 0.9590518305580027, 0.9564861585032663, 0.9570542361201553, 0.9559709675414783, 0.9576020389075833, 0.9572782607690481]
R2 Test List:  [0.4493670223405215, 0.5059905053979984, 0.44399920386206815, 0.45567427779818803, 0.44580636615743585, 0.47064638015968063, 0.4906937636783618, 0.49667774049189606, 0.4190553830596295, 0.4730306606677738]
r2_train_list len: 10
r2_test_list len: 10


***Let's run the regular hyena-dna embeds with the extra features as well as hyena-dna embeds from my fine-tuned weights***

In [18]:
# Running the regular hyena-dna embeddings (no-fine-tuning, but with extra features)
hyena_embeds_list = glob_files('UTR_regions')
print('Hyena embeds list: ', hyena_embeds_list)

run_cross_validation_for_files(hyena_embeds_list, model_list, model_dict, y_np, n_folds, dataset='UTR_regions')

Hyena embeds list:  ['./hyena_embeds/reshaped_min_max_hyena.npy', './hyena_embeds/reshaped_average_hyena.npy', './hyena_embeds/pca_components_2.npy', './hyena_embeds/pca_components_5.npy', './hyena_embeds/pca_components_10.npy', './hyena_embeds/pca_components_150.npy', './hyena_embeds/pacmap_components_2.npy', './hyena_embeds/pacmap_components_5.npy', './hyena_embeds/pacmap_components_10.npy', './hyena_embeds/pacmap_components_100.npy']
Hyena embeds list:  ['./hyena_embeds/reshaped_min_max_hyena.npy', './hyena_embeds/reshaped_average_hyena.npy', './hyena_embeds/pca_components_2.npy', './hyena_embeds/pca_components_5.npy', './hyena_embeds/pca_components_10.npy', './hyena_embeds/pca_components_150.npy', './hyena_embeds/pacmap_components_2.npy', './hyena_embeds/pacmap_components_5.npy', './hyena_embeds/pacmap_components_10.npy', './hyena_embeds/pacmap_components_100.npy']
File Name:  ./hyena_embeds/reshaped_min_max_hyena.npy
Head of ds:                                              sequenc

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.29677301174193815, 0.2939086054200274, 0.2919545195289268, 0.2867600024396463, 0.29354869838304143, 0.2933344248221257, 0.29372500538188007, 0.28964259405851756, 0.2948588818287483, 0.29217201155979944]
R2 Test List:  [0.201512478244271, 0.22404360688033143, 0.24538974058582608, 0.296448457990234, 0.22886258819900818, 0.23388498329911245, 0.230266800112921, 0.26894704161591765, 0.22144720857941336, 0.24311315027776947]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Le

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9991089528316504, 0.9992015867617514, 0.9991191087658404, 0.9990742151012281, 0.9991794544626308, 0.9991210155565495, 0.9990960095522817, 0.9991652871660265, 0.9991432584188391, 0.9990922707642256]
R2 Test List:  [0.3596582417160654, 0.3594176737730834, 0.36002405739537424, 0.3828910091557517, 0.3464569627148305, 0.37851210786154155, 0.3392698633623109, 0.3614830806038387, 0.33604119136551436, 0.34483234454729905]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./hyena_embeds/reshaped_average_hyena.npy
Head of ds:                                              sequence       tpm
0  GGGGACCAGGTGCCGTAAGGTGCGGCTGGCCCAATGTGCGCCTATG... -1.226293
1  GGGGACCAGGTGCCGTAAGCCATGAATTAATGAATATCTTTACTTA...  0.086170
2  GGGGACCAGGTGCCGTAAGGTCACGCTTACATTCACGCCCTCCTCC... -1.842423
3  GGGGACCAGGTGCCGTAAGTTGCTATGCAGATGCTTTATACTTCTT...  0.371952
4  GGGGACCAGGTGCCGTAAGCATTGTTAGCTTTTTATGCATTATAAT... -1.729185
Head of df_opt_seqs:            

...

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.8810618795875398, 0.8805600775756375, 0.8755833737582576, 0.8782475351465007, 0.8812306785392969, 0.8812245575875696, 0.8763021994084551, 0.8758539127120022, 0.8791797240183972, 0.8780197809744649]
R2 Test List:  [0.2923550514102984, 0.35442661615791504, 0.32698593233924966, 0.33033155227290556, 0.3353431781284182, 0.31650514311404676, 0.2768417404547948, 0.3322459581502478, 0.2898146950278424, 0.33981338109083303]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./hyena_embeds/pacmap_components_100.npy
Head of ds:                                              sequence       tpm
0  GGGGACCAGGTGCCGTAAGGTGCGGCTGGCCCAATGTGCGCCTATG... -1.226293
1  GGGGACCAGGTGCCGTAAGCCATGAATTAATGAATATCTTTACTTA...  0.086170
2  GGGGACCAGGTGCCGTAAGGTCACGCTTACATTCACGCCCTCCTCC... -1.842423
3  GGGGACCAGGTGCCGTAAGTTGCTATGCAGATGCTTTATACTTCTT...  0.371952
4  GGGGACCAGGTGCCGTAAGCATTGTTAGCTTTTTATGCATTATAAT... -1.729185
Head of df_opt_seqs:            

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.26881394500447675, 0.266492447591413, 0.2656754679466836, 0.2586634250288785, 0.26324534224735274, 0.2601967089802154, 0.2677108135649032, 0.2659877775342364, 0.26566495611317364, 0.2619581287236804]
R2 Test List:  [0.21350869969436337, 0.22994001028250066, 0.24065233947862497, 0.3046854179254801, 0.2635032444369656, 0.29191387700371985, 0.2243484650773655, 0.24122036932533042, 0.2420263223169249, 0.27498915334316276]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Len

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.939583264442027, 0.9437929709597863, 0.9402397073828691, 0.938798523983521, 0.9425982860771781, 0.9416635409452613, 0.9424504750800438, 0.9405853639248088, 0.9437570689395727, 0.9424703423832504]
R2 Test List:  [0.32321790818878493, 0.3196125323260607, 0.3014023137056444, 0.35771081022729845, 0.2921137876013551, 0.26660744077133014, 0.241548118807552, 0.3321809627998602, 0.3011896267265525, 0.3500753457869836]
r2_train_list len: 10
r2_test_list len: 10


In [19]:
# Running the fine-tuned hyena-dna embeddings with extra features
hyena_embeds_list = glob_files('fine_tuned_pe')
print('Hyena embeds list: ', hyena_embeds_list)

run_cross_validation_for_files(hyena_embeds_list, model_list, model_dict, y_np, n_folds, dataset='fine_tuned_pe')

Hyena embeds list:  ['./fine_tuned_hyena_embeds/reshaped_min_max_hyena.npy', './fine_tuned_hyena_embeds/reshaped_average_hyena.npy', './fine_tuned_hyena_embeds/pca_components_2.npy', './fine_tuned_hyena_embeds/pca_components_5.npy', './fine_tuned_hyena_embeds/pca_components_10.npy', './fine_tuned_hyena_embeds/pca_components_150.npy', './fine_tuned_hyena_embeds/pacmap_components_2.npy', './fine_tuned_hyena_embeds/pacmap_components_5.npy', './fine_tuned_hyena_embeds/pacmap_components_10.npy', './fine_tuned_hyena_embeds/pacmap_components_100.npy']
Hyena embeds list:  ['./fine_tuned_hyena_embeds/reshaped_min_max_hyena.npy', './fine_tuned_hyena_embeds/reshaped_average_hyena.npy', './fine_tuned_hyena_embeds/pca_components_2.npy', './fine_tuned_hyena_embeds/pca_components_5.npy', './fine_tuned_hyena_embeds/pca_components_10.npy', './fine_tuned_hyena_embeds/pca_components_150.npy', './fine_tuned_hyena_embeds/pacmap_components_2.npy', './fine_tuned_hyena_embeds/pacmap_components_5.npy', './fine

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.3390844172884173, 0.3334079421993311, 0.3370914111573564, 0.33657153357506087, 0.3370466628667056, 0.3377439539574417, 0.33729322646324367, 0.3373025443667973, 0.33416928492240683, 0.33441375940519413]
R2 Test List:  [0.2614555223444708, 0.31067480572568096, 0.27654563352950734, 0.2878928708681485, 0.2737730972087362, 0.2690464005759847, 0.27555634141112295, 0.2835556027180397, 0.30536053764131, 0.30655739198991927]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Lengt

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9991695896328922, 0.9991027025274039, 0.9989934005648884, 0.9991504257705167, 0.9992069416997517, 0.9991391995198291, 0.9991044903190895, 0.9991379013116329, 0.9990072225174391, 0.9991663860877649]
R2 Test List:  [0.4458458281979861, 0.4786306964109286, 0.44621610103909337, 0.4478988743108069, 0.4490150275959093, 0.44701378602088115, 0.4229220953558338, 0.4361886218290809, 0.4567060931629643, 0.4898259570211213]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./fine_tuned_hyena_embeds/reshaped_average_hyena.npy
Head of ds:                                              sequence       tpm
0  GGGGACCAGGTGCCGTAAGGTGCGGCTGGCCCAATGTGCGCCTATG... -1.226293
1  GGGGACCAGGTGCCGTAAGCCATGAATTAATGAATATCTTTACTTA...  0.086170
2  GGGGACCAGGTGCCGTAAGGTCACGCTTACATTCACGCCCTCCTCC... -1.842423
3  GGGGACCAGGTGCCGTAAGTTGCTATGCAGATGCTTTATACTTCTT...  0.371952
4  GGGGACCAGGTGCCGTAAGCATTGTTAGCTTTTTATGCATTATAAT... -1.729185
Head of df_opt_seqs:   

...

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9084475451723395, 0.9060075491194862, 0.9071629497545851, 0.9087873835201086, 0.9085509356836629, 0.9111573013571194, 0.9072127886467516, 0.903777146379934, 0.9094448139217076, 0.9059094628902195]
R2 Test List:  [0.41306970054185355, 0.5107778628985606, 0.43823086159538993, 0.4690814857675448, 0.42624174845972596, 0.4494992131847807, 0.4639350157240417, 0.4857919183210869, 0.44622281252137985, 0.49594131564264576]
r2_train_list len: 10
r2_test_list len: 10
File Name:  ./fine_tuned_hyena_embeds/pacmap_components_100.npy
Head of ds:                                              sequence       tpm
0  GGGGACCAGGTGCCGTAAGGTGCGGCTGGCCCAATGTGCGCCTATG... -1.226293
1  GGGGACCAGGTGCCGTAAGCCATGAATTAATGAATATCTTTACTTA...  0.086170
2  GGGGACCAGGTGCCGTAAGGTCACGCTTACATTCACGCCCTCCTCC... -1.842423
3  GGGGACCAGGTGCCGTAAGTTGCTATGCAGATGCTTTATACTTCTT...  0.371952
4  GGGGACCAGGTGCCGTAAGCATTGTTAGCTTTTTATGCATTATAAT... -1.729185
Head of df_opt_seqs:  

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.4185705600517443, 0.41328463149343675, 0.41765289163767194, 0.41133128017266274, 0.41984927341035216, 0.41243395337406863, 0.41541007264179364, 0.4140483957888076, 0.41765727401511044, 0.4099212083646442]
R2 Test List:  [0.37309089183641575, 0.41943270402022814, 0.37955598344503483, 0.4387900389702064, 0.3566330022894366, 0.42860981158684974, 0.40231996124906155, 0.41525105094985193, 0.38132513214373476, 0.45054166389283534]
r2_train_list len: 10
r2_test_list len: 10
Running model:  xgb
Model name and hyperparams:  xgb {'learning_rate': 0.22, 'max_depth': 10, 'reg_lambda': 1.8, 'reg_alpha': 0.89, 'n_jobs': -1}
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1045
Length of X-train fold:  9402
Length of X-test fold:  1

/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype

r2_train_list len: 10
r2_test_list len: 10
R2 Train List:  [0.9576898730771394, 0.95803952505129, 0.9588792275015458, 0.9542446515563556, 0.9598287721706187, 0.9590934495881717, 0.9585577923943284, 0.9551241812835609, 0.958735957648332, 0.9591231499670899]
R2 Test List:  [0.43172398061719697, 0.48773060772810717, 0.43689451692418235, 0.4626155626646663, 0.4430972553577589, 0.4770837934402328, 0.4857915342817597, 0.49541350211008284, 0.4424033592126896, 0.4848627988092412]
r2_train_list len: 10
r2_test_list len: 10


# Now lets use Optuna to tune our hyperparameters :)

In [34]:
# Objective function for optimization
def objective(trial, X, y):
    classifier_name = trial.suggest_categorical('classifier', ['XGBoost', 'LinearRegression', 'Ridge', 'RandomForest'])
    
    if classifier_name == 'XGBoost':
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 1, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
            'n_jobs': -1
        }
        clf = XGBRegressor(**xgb_params)

    elif classifier_name == 'LinearRegression':
        clf = LinearRegression(n_jobs=-1)

    elif classifier_name == 'Ridge':
        ridge_params = {
            'alpha': trial.suggest_float('alpha', 0.1, 10.0)
        }
        clf = Ridge(**ridge_params)

    elif classifier_name == 'RandomForest':
        rf_params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_int('max_depth', 1, 20),
        }
        clf = RandomForestRegressor(**rf_params, n_jobs=-1)
    
    # return cross_val_score(clf, X, y, n_jobs=-1, cv=3).mean()
    return cross_validate(clf, X, y, cv=4, n_jobs=-1, return_train_score=True)['test_score'].mean() # Use this instead, I believe the cross_val_score was returning training scores instead of test. You are supposed to tune hyperparameters based on the test!!!

def optimize_hyperparameters(file_list):
    embeddings_file_list = file_list
    results = {}

    label_df = smart_read_csv(LABEL_DATA_PATH)
    y = label_df.iloc[:, 1]
    y_np = y.to_numpy()
    print('y shape: ', y.shape)
    
    for file in embeddings_file_list:
        print(f'\nFile being processed in hyperparameter optimization:\n {file}')
        embeddings = np.load(file)
        # Push to CPU
        X = embeddings
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, X, y), n_trials=10) # Change n_trials to however many trials you have time for!

        results[file] = {
            "Value": study.best_value,
            "Params": study.best_params
        }

    return results

# Example of using the function
# file_path = "*.csv"  # Adjust to the actual path
# results = optimize_hyperparameters(file_path)
# print(results)

def save_results_to_json(results, dataset_name, file_path):
    # Creating the filename with the dataset name
    filename = f"{file_path}optuna_results_{dataset_name}.json"

    # Saving the results dictionary to a JSON file
    with open(filename, 'w') as json_file:
        json.dump(results, json_file)

    print(f"Results saved to {filename}")

def optimize_dataset(dataset_name):
    if dataset_name == 'no_augment':
        path = './hyena_embeds/'
        globbed_reduced_embeds_path = glob.glob(f'{path}*.npy')
        results = optimize_hyperparameters(globbed_reduced_embeds_path)
        save_results_to_json(results, dataset_name, path)
        
    elif dataset_name == 'UTR_regions':
        path = './hyena_embeds_pe/'
        globbed_reduced_embeds_path = glob.glob(f'{path}*.npy')
        results = optimize_hyperparameters(globbed_reduced_embeds_path)
        save_results_to_json(results, dataset_name, path)
        
    elif dataset_name == 'fine_tuned':
        path = './fine_tuned_hyena_embeds/'
        globbed_reduced_embeds_path = glob.glob(f'{path}*.npy')
        results = optimize_hyperparameters(globbed_reduced_embeds_path)
        save_results_to_json(results, dataset_name, path)
        
    elif dataset_name == 'fine_tuned_pe':
        path = './fine_tuned_hyena_embeds_pe/'
        globbed_reduced_embeds_path = glob.glob(f'{path}*.npy')
        results = optimize_hyperparameters(globbed_reduced_embeds_path)
        save_results_to_json(results, dataset_name, path)
        
    else:
        raise ValueError(f'Invalid dataset: {dataset_name}')

In [36]:
# Run optimize_dataset for each dataset
# for dataset_name in ['UTR_regions', 'no_augment', 'fine_tuned', 'fine_tuned_pe']:
#     optimize_dataset(dataset_name)

optimize_dataset('fine_tuned') # 10/26/23, THIS DOESN'T WORK FOR 'UTR_Regions' OR 'fine_tuned_pe' datasets, since the glob function isn't grabbing the right files

[I 2023-11-13 16:49:28,327] A new study created in memory with name: no-name-96f70fd3-3a97-4eec-9cc8-49db96bdaae5


y shape:  (7239,)

File being processed in hyperparameter optimization:
 ./fine_tuned_hyena_embeds/reshaped_min_max_hyena.npy


[I 2023-11-13 16:49:30,627] Trial 0 finished with value: 0.3753377182772739 and parameters: {'classifier': 'LinearRegression'}. Best is trial 0 with value: 0.3753377182772739.
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be remove


File being processed in hyperparameter optimization:
 ./fine_tuned_hyena_embeds/reshaped_average_hyena.npy


[I 2023-11-13 16:59:30,054] Trial 0 finished with value: 0.3791550317540814 and parameters: {'classifier': 'Ridge', 'alpha': 5.126251839464566}. Best is trial 0 with value: 0.3791550317540814.
[I 2023-11-13 16:59:30,945] Trial 1 finished with value: 0.598288769906769 and parameters: {'classifier': 'LinearRegression'}. Best is trial 1 with value: 0.598288769906769.
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstanc

...


File being processed in hyperparameter optimization:
 ./fine_tuned_hyena_embeds/pacmap_components_10.npy


[I 2023-11-13 17:07:47,628] Trial 3 finished with value: 0.48289923864417234 and parameters: {'classifier': 'RandomForest', 'n_estimators': 129, 'max_depth': 2}. Best is trial 3 with value: 0.48289923864417234.
[I 2023-11-13 17:07:48,422] Trial 4 finished with value: 0.5705907733564839 and parameters: {'classifier': 'RandomForest', 'n_estimators': 121, 'max_depth': 3}. Best is trial 4 with value: 0.5705907733564839.
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and 


File being processed in hyperparameter optimization:
 ./fine_tuned_hyena_embeds/pacmap_components_100.npy


[I 2023-11-13 17:07:52,495] Trial 1 finished with value: 0.5283961941575563 and parameters: {'classifier': 'Ridge', 'alpha': 2.360087751784299}. Best is trial 0 with value: 0.572024237567136.
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/zbates/hyena-dna/env/lib/python3.10/site-packages/xgboost/data.py:520: FutureWarning: is_sparse is deprecated an

Results saved to ./fine_tuned_hyena_embeds/optuna_results_fine_tuned.json


# Lets use the optimal configuration and test the val ds

**The below is going to call a separate function to take the top performer from Optuna, and test it on the Validation dataset**

In [37]:
from optimal_model_test import *

In [40]:
# THIS COULD BE AUTOMATED BY ADDING A SETTINGS.PY FILE
json_file_path = './fine_tuned_hyena_embeds/optuna_results_fine_tuned.json'
model, r2, mse = run_pipeline(json_file_path, y_train_path=LABEL_DATA_PATH, X_val_path=os.path.join(PATH_TO_GENERATED_EMBEDDINGS, 'val_40.pth'), y_val_path='./data/train_val_test_splits/val.csv', specific_model=None)

X shape: (1035, 19456)


In [41]:
print(f'The model, r2, and mse are: {model}, {r2}, {mse}')

The model, r2, and mse are: Ridge(alpha=9.733764055482645), 0.5341146568734052, 0.6174119691903348
